In [ ]:
%pylab inline

import warnings
warnings.filterwarnings("ignore")

from SCCAF import *
import os
os.chdir("./RETINA/")


In [ ]:
def sele_dataset(adTr, adLn, n_HVG = 2000, Ln_species = 'Patch', Tr_species = 'TenX'):
#generate new dataset with HVG only
#input adTr: trainging set, full dataset; adLn:testing set, full dataset; n_HVG, number of HVG
# merker_gene: use only marker gene
#output Tr_sub, Ln_sub: splited Training and testing dataset, with combine calculated PCA
   
    
    adTr.X = adTr.layers['raw']
    #sc.pp.scale(adTr)
    
    adLn.X = adLn.layers['raw']
    #sc.pp.scale(adLn)
    
    if scipy.sparse.issparse(adTr.X):
        adTr_selected_gene = adTr.var_names[np.array(np.std(adTr.X.todense(), axis=0).argsort())[0][-n_HVG:][::-1]]
    else:
        adTr_selected_gene = adTr.var_names[np.array(np.std(adTr.X, axis=0).argsort())[-n_HVG:][::-1]]
    
    
    if scipy.sparse.issparse(adLn.X):
        adLn_selected_gene = adLn.var_names[np.array(np.std(adLn.X.todense(), axis=0).argsort())[0][-n_HVG:][::-1]]
    else:
        adLn_selected_gene = adLn.var_names[np.array(np.std(adLn.X, axis=0).argsort())[-n_HVG:][::-1]]
    
    select_gene  = unique(list(adTr_selected_gene)+list(adLn_selected_gene))
    
    adTr_sele = adTr[:,select_gene]
    adTr_sele.obs['source']= Tr_species
    
    adLn_sele = adLn[:,select_gene]
    adLn_sele.obs['source']= Ln_species

    TrLn = adTr_sele.concatenate(adLn_sele)
    

    #sc.pp.scale(TrLn)
    #sc.tl.pca(TrLn)
    #sc.pl.pca(TrLn, color = ['marker1','source'])
    
    #Tr_sub = TrLn[TrLn.obs['source']==Tr_source,:]
    #Ln_sub = TrLn[TrLn.obs['source']==Ln_source,:]
     
    return TrLn

In [ ]:
def run_PCA_Harmony(TrLn, run_Harmony = True, theta = 5,rep = 1,key = 'org_cluster' ):
#run PCA and Harmony (optional) 
#input TrLn, combined dataset from Training set and testing set, generated by function sele_dataset
#output Tr_sub, Ln_sub: splited Training and testing dataset, with combine calculated PCA and Harmony, in layer obsm
    
    #TrLn.X = TrLn.layers['raw']
    if scipy.sparse.issparse(TrLn.X): 
        X = TrLn.X.todense().T
    else:
        X = TrLn.T
    #TrLn.obsm['X_pca'] = sc.tl.pca(np.array(scale(X.T)), n_comps=100)
    #sc.pp.scale(TrLn)
    sc.tl.pca(TrLn)
    #sc.pl.pca(TrLn, color = [key,'source'])
    
    #Tr_sub = TrLn[TrLn.obs['source']==Tr_source,:]
    #Ln_sub = TrLn[TrLn.obs['source']==Ln_source,:]
    
    if run_Harmony:
        
        data_mat = TrLn.obsm['X_pca']
        meta_data = TrLn.obs

        TrLn.obsm['X_pca'].shape
        #ho = hm.run_harmony(data_mat, meta_data, 'species')
        ho = hm.run_harmony(data_mat, meta_data, 'source',theta = theta)
        res = ho.Z_corr
        print('Run Harmony')
        TrLn.obsm['X_harmony'] = res.T
        sc.pp.neighbors(TrLn)
        #sc.tl.umap(TrLn)
        #sc.tl.tsne(humk, n_jobs = 20)
        #sc.pl.umap(TrLn, color = ['source',key])
        #sc.pl.tsne(humk,color = ['species','marker1','marker2'])

#         for i in range(1,rep):
        
#             print("this is the " + str(i) + " time of harmony" )
    
#             data_mat = TrLn.obsm['X_harmony']
#             meta_data = TrLn.obs

#             #ho = hm.run_harmony(data_mat, meta_data, 'species')
#             ho = hm.run_harmony(data_mat, meta_data, 'source', theta = 5)
#             res = ho.Z_corr
#             TrLn.obsm['X_harmony'] = res.T
#             sc.pp.neighbors(TrLn,use_rep = 'X_harmony')
#             print('Finished Harmony')
#             #sc.tl.umap(TrLn)
#             #sc.tl.tsne(humk, n_jobs = 20)
#             #sc.pl.umap(TrLn, color = ['source',key])
#             #sc.pl.tsne(humk,color = ['species','marker1','marker2'])
 
    
    return TrLn

In [ ]:
def get_prob_mean(df_prob_rep):
#caluculating probability mean from the big repeat df
#input: pd.Series each obj is a prob dataframe of each repeat
#output: df with mean of each cell, each prob 
    cell_type_list = list(df_prob_rep[0].columns)
    df_prob_mean = pd.DataFrame()
    for cell_type in cell_type_list:
        #print('calculating ' + cell_type + '...')
        df_prob_type = pd.DataFrame()
        for rep_ind in list(df_prob_rep.index):
            #print(rep_ind)
            df_prob_type_mean = pd.DataFrame()
            df_prob_type[rep_ind]=df_prob_rep[rep_ind][cell_type]
        df_prob_type_mean = mean(df_prob_type, axis = 1)
        df_prob_mean[cell_type]=df_prob_type_mean
    return df_prob_mean

In [ ]:
def get_prob_median(df_prob_rep):
#caluculating probability median from the big repeat df
#input: pd.Series each obj is a prob dataframe of each repeat
#output: df with median of each cell, each prob 
    cell_type_list = list(df_prob_rep[0].columns)
    df_prob_median = pd.DataFrame()
    for cell_type in cell_type_list:
        #print('calculating ' + cell_type + '...')
        df_prob_type = pd.DataFrame()
        for rep_ind in list(df_prob_rep.index):
            #print(rep_ind)
            df_prob_type_median = pd.DataFrame()
            df_prob_type[rep_ind]=df_prob_rep[rep_ind][cell_type]
        df_prob_type_median = median(df_prob_type, axis = 1)
        df_prob_median[cell_type]=df_prob_type_median
    return df_prob_median

In [ ]:
def get_prob_max(df_prob_rep):
#caluculating probability max from the big repeat df
#input: pd.Series each obj is a prob dataframe of each repeat
#output: df with max of each cell, each prob 
    cell_type_list = list(df_prob_rep[0].columns)
    df_prob_max = pd.DataFrame()
    for cell_type in cell_type_list:
        #print('calculating ' + cell_type + '...')
        df_prob_type = pd.DataFrame()
        for rep_ind in list(df_prob_rep.index):
            #print(rep_ind)
            df_prob_type_max = pd.DataFrame()
            df_prob_type[rep_ind]=df_prob_rep[rep_ind][cell_type]
        df_prob_type_max = df_prob_type.max(axis = 1)
        df_prob_max[cell_type]=df_prob_type_max
    return df_prob_max

In [ ]:
def get_prob_min(df_prob_rep):
#caluculating probability min from the big repeat df
#input: pd.Series each obj is a prob dataframe of each repeat
#output: df with min of each cell, each prob 
    cell_type_list = list(df_prob_rep[0].columns)
    df_prob_min = pd.DataFrame()
    for cell_type in cell_type_list:
        #print('calculating ' + cell_type + '...')
        df_prob_type = pd.DataFrame()
        for rep_ind in list(df_prob_rep.index):
            #print(rep_ind)
            df_prob_type_min = pd.DataFrame()
            df_prob_type[rep_ind]=df_prob_rep[rep_ind][cell_type]
        df_prob_type_min = df_prob_type.min(axis = 1)
        df_prob_min[cell_type]=df_prob_type_min
    return df_prob_min

In [ ]:
def data_subset(adTr,key = 'org_cluster', n=50, frac=1):
#subsetting the datasets based on their cell types
    X_train, X_test, y_train, y_test = train_test_split_per_type(adTr.X, adTr.obs[key], n=n, frac=frac)
    adTr_sub = adTr[y_train.index,:]
    return adTr_sub

In [ ]:
def find_save_score(df_prob_rep, manual_label, param, savepath = 'data/retina_label/score_summary.csv'):
#recored the score and write to csv
#input df_prob_rep, repeating results
#input manual_label: manual labeling results of each patched cells
#input param, the parameter string indicating 
#outout, score results
    f = open(f'data/retina_label/predict_results/{str(param)}.pckl', 'wb')
    pickle.dump(df_prob_rep, f)
    f.close()
    
    #get mean, median and min and max from each repeats
    df_prob_median = pd.DataFrame()
    df_prob_median = get_prob_median(df_prob_rep)
    df_prob_median.index = df_prob_rep['rep_0'].index
    
    df_prob_mean = pd.DataFrame()
    df_prob_mean = get_prob_mean(df_prob_rep)
    df_prob_mean.index = df_prob_rep['rep_0'].index
    
    df_prob_max = pd.DataFrame()
    df_prob_max = get_prob_max(df_prob_rep)
    df_prob_max.index = df_prob_rep['rep_0'].index
    
    df_prob_min = pd.DataFrame()
    df_prob_min = get_prob_min(df_prob_rep)
    df_prob_min.index = df_prob_rep['rep_0'].index
    
    #find the suited cluster
    pre_results = pd.DataFrame(index = df_prob_median.index)
    pre_results['predict_cluster_median'] = df_prob_median.idxmax(axis=1)
    pre_results['predict_cluster_medianprob'] = df_prob_median.max(axis=1)
    
    pre_results['predict_cluster_mean'] = df_prob_mean.idxmax(axis=1)
    pre_results['predict_cluster_meanprob'] = df_prob_mean.max(axis=1)
    
    pre_results['predict_cluster_max'] = df_prob_max.idxmax(axis=1)
    pre_results['predict_cluster_maxprob'] = df_prob_max.max(axis=1)
    
    pre_results['predict_cluster_min'] = df_prob_min.idxmin(axis=1)
    pre_results['predict_cluster_minprob'] = df_prob_min.min(axis=1)
    
    # get each category
    NotChange_list = manual_label[manual_label['allow_modification']=='N'].index.tolist()
    MaybeChange_list = manual_label[manual_label['allow_modification']=='*'].index.tolist()
    Change_list = manual_label[manual_label['allow_modification']=='Y'].index.tolist()
    
    #compare annotation
    pre_results['conclusion'] = manual_label['conclusion']
    pre_results['Match_median'] = pre_results['conclusion']== pre_results['predict_cluster_median']
    pre_results['Match_mean'] = pre_results['conclusion']== pre_results['predict_cluster_mean']
    pre_results['Match_max'] = pre_results['conclusion']== pre_results['predict_cluster_max']
    pre_results['Match_min'] = pre_results['conclusion']== pre_results['predict_cluster_min']
    
    score_sum = pd.DataFrame()
    score_sum.loc[0,'Param'] = param

    #.sum() get the number of True    
    score_sum.loc[0,'total_median'] = pre_results.loc[:,'Match_median'].sum()/len(pre_results)
    score_sum.loc[0,'total_mean'] = pre_results.loc[:,'Match_mean'].sum()/len(pre_results)
    score_sum.loc[0,'total_max'] = pre_results.loc[:,'Match_max'].sum()/len(pre_results)
    #score_sum.loc[0,'total_min'] = pre_results.loc[:,'Match_min'].sum()/len(pre_results)

    score_sum.loc[0,'NotChange_median'] =  pre_results.loc[NotChange_list,'Match_median'].sum()/len(NotChange_list)
    score_sum.loc[0,'NotChange_mean'] =  pre_results.loc[NotChange_list,'Match_mean'].sum()/len(NotChange_list)
    score_sum.loc[0,'NotChange_max'] =  pre_results.loc[NotChange_list,'Match_max'].sum()/len(NotChange_list)
    #score_sum.loc[0,'NotChange_min'] =  pre_results.loc[NotChange_list,'Match_min'].sum()/len(NotChange_list)
    
    score_sum.loc[0,'MaybeChange_median'] =  pre_results.loc[MaybeChange_list,'Match_median'].sum()/len(MaybeChange_list)
    score_sum.loc[0,'MaybeChange_mean'] =  pre_results.loc[MaybeChange_list,'Match_mean'].sum()/len(MaybeChange_list)
    score_sum.loc[0,'MaybeChange_max'] =  pre_results.loc[MaybeChange_list,'Match_max'].sum()/len(MaybeChange_list)
    #score_sum.loc[0,'MaybeChange_min'] =  pre_results.loc[MaybeChange_list,'Match_min'].sum()/len(MaybeChange_list)

    score_sum.loc[0,'Change_median'] =  pre_results.loc[Change_list,'Match_median'].sum()/len(Change_list)
    score_sum.loc[0,'Change_mean'] =  pre_results.loc[Change_list,'Match_mean'].sum()/len(Change_list)
    score_sum.loc[0,'Change_max'] =  pre_results.loc[Change_list,'Match_max'].sum()/len(Change_list)
    #score_sum.loc[0,'Change_min'] =  pre_results.loc[Change_list,'Match_min'].sum()/len(Change_list)

    
    import csv   
    fields=list(score_sum.loc[0])
    with open(savepath, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)
    
    return(score_sum, pre_results)


In [ ]:
def label_projection (adTr, adLn, key = 'org_cluster', mode = 'X', 
                      n_HVG = 100, n=50, frac=0.8, 
                      Ln_species = 'Patch', Tr_species = 'TenX',
                      plot_tsne = False):
# SCCAF projectin
#input: adTr: training set; adLn, testing set;key, label for learning; mode, X or PCA or harmony PCA
#output: dataframe of probablity of each cell, each label
    
    adTr_sub = data_subset(adTr)
    TrLn = sele_dataset(adTr_sub, adLn, n_HVG = n_HVG,Ln_species = Ln_species, Tr_species = Tr_species)
    print('get dataset')
    
    if mode == 'X':
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        Tr_matrix = Tr_sele.X
        Ln_matrix = Ln_sele.X
        print('split dataset')
        
    elif mode == 'X_pca':
          

        TrLn = run_PCA_Harmony(TrLn, run_Harmony = False, theta = 5,rep = 3)
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        
        if scipy.sparse.issparse(adTr.obsm['X_pca']): 
            Tr_matrix  = Tr_sele.obsm['X_pca'].todense()
        else:
            Tr_matrix  = Tr_sele.obsm['X_pca']
            
        if scipy.sparse.issparse(adLn.obsm['X_pca']): 
            Ln_matrix  = Ln_sele.obsm['X_pca'].todense()
        else:
            Ln_matrix  = Ln_sele.obsm['X_pca']
            
        
    elif mode == 'X_harmony':    
        adTr_sub = data_subset(adTr)
        TrLn = sele_dataset(adTr_sub, adLn, n_HVG = n_HVG,Ln_species = Ln_species, Tr_species = Tr_species)  
        
        TrLn = run_PCA_Harmony(TrLn, run_Harmony = True, theta = 5,rep = 4)
        Tr_sele = TrLn[TrLn.obs['source']==Tr_species,:]
        Ln_sele = TrLn[TrLn.obs['source']==Ln_species,:]
        
        if scipy.sparse.issparse(Tr_sele.obsm['X_harmony']): 
            Tr_matrix  = Tr_sele.obsm['X_harmony'].todense()
        else:
            Tr_matrix  = Tr_sele.obsm['X_harmony']
            
        if scipy.sparse.issparse(Ln_sele.obsm['X_harmony']): 
            Ln_matrix  = Ln_sele.obsm['X_harmony'].todense()
        else:
            Ln_matrix  = Ln_sele.obsm['X_harmony']
        
    print('split learning')    
    X_train, X_test, y_train, y_test = train_test_split_per_type(Tr_matrix, Tr_sele.obs[key], n=n, frac=frac)
    
    print('regression')
    clf = LogisticRegression(random_state=1, penalty='l1', C=0.5, solver = 'saga')
    
    clf.fit(X_train, y_train)

    if plot_tsne:
        sc.pl.umap(adTr, color = [key])

    Tr_sele.obs[key+'_predict'] = clf.predict(Tr_matrix)
    if plot_tsne:
        sc.pl.tsne(adTr, color = [key,key+'_predict'])
    print('predicting')
    adLn.obs[key + '_predict'] = clf.predict(Ln_matrix)
    #sc.pl.tsne(adLn, color = [key,key+'_predict'])
    df_prob = pd.DataFrame(clf.predict_proba(Ln_matrix),index = adLn.obs_names, columns = clf.classes_)
    
    return df_prob

In [ ]:
#load 10X
ad10x = sc.read('data/TenX_atlas.h5')


In [ ]:
#check_results
sc.pl.umap(ad10x, color = 'org_cluster')

In [ ]:
#data pre_processing
figsize(4,4)
ad10x.X = ad10x.layers['counts']
sc.pp.normalize_per_cell(ad10x, counts_per_cell_after=1e4)
sc.pp.log1p(ad10x)
ad10x.layers['raw'] = ad10x.X
ad10x.X

In [ ]:
#load patch-seq data
adpatch = sc.read('data/retina_patch_all_qc.h5') 

In [ ]:
#patch-seq data preprocessing
figsize(4,4)
adpatch.X = adpatch.layers['counts']
sc.pp.normalize_per_cell(adpatch, counts_per_cell_after=1e5)
sc.pp.log1p(adpatch)
adpatch.layers['raw'] = adpatch.X
adpatch.X

In [ ]:
#check common genes
common_genes = list((set(ad10x.var_names.tolist()))&(set(adpatch.var_names.tolist())))
len(common_genes)

In [ ]:
#use only the genes shared by the two datasets
ad10x = ad10x[:,common_genes]
ad10x
adpatch = adpatch[:,common_genes]
adpatch

In [ ]:
#Looping of SCCAF using different parameters

n_gene_list  = [2000,4000,6000,8000,10000]
mode_list = ['X','X_pca','X_harmony']
tenx_cell_list = [25, 50, 75, 100,200,1000]
N_rep_list =[1]
penelty_list = ['l1','l2']
C_value_list = [0.3,0.5,0.8]
solver_mode_list = ['saga']


for n_gene in n_gene_list:
    for mode_use in mode_list:
        for tenx_cell in tenx_cell_list:
            for N_rep in N_rep_list:
                for penelty in penelty_list:
                    for C_value in C_value_list:
                        for solver_mode in solver_mode_list:
                            param = f'HVG_{n_gene}_{mode_use}_Tenx_{tenx_cell}_rep_{N_rep}_pen_{penelty}_C_{C_value}_solver_{solver_mode}'
                            print(param)
                            
                            #skip calculated ones
                            finished = pd.read_csv('data/score_summary.csv')
                            if param in list(finished['Param']): 
                                    print('finished')
                                    continue
                            
                            #load data
                            ad10X, adpatch = load_h5(path_tenx = 'data/onc_atlas.h5', 
                                            path_patch = 'data/retina_patch_all_qc.h5' )
                            
                            #start of the function
                            df_prob_rep = pd.Series()
                            for rep in range(0, N_rep):
                                print('repeat ' + str(rep))
                                
                                df_prob = pd.DataFrame()
                                df_prob = label_projection(adTr = ad10x, adLn = adpatch,key = 'org_cluster',  Tr_species = 'TenX',Ln_species = 'Patch', 
                                                          mode = mode_use, n_HVG = n_gene, n=tenx_cell, frac=0.8,
                                                           panelty = penelty, C_value = C_value, solver_mode = solver_mode,)
                                df_prob_rep['rep_'+str(rep)] = df_prob
                            
                            #calucate score and write to csv
                            score_sum, pre_results = find_save_score(df_prob_rep, manual_label, param)

# summary_all parameters

In [ ]:

n_gene_list  = [2000,4000,6000,8000,10000]
mode_list = ['X','X_pca','X_harmony']
tenx_cell_list = [25, 50, 100,200,1000]
N_rep_list =[1]
penelty_list = ['l1','l2']
C_value_list = [0.3,0.5,0.8]
solver_mode_list = ['saga']


para_list = []
for n_gene in n_gene_list:
    for mode_use in mode_list:
        for tenx_cell in tenx_cell_list:
            for N_rep in N_rep_list:
                for penelty in penelty_list:
                    for C_value in C_value_list:
                        for solver_mode in solver_mode_list:
                            param = f'HVG_{n_gene}_{mode_use}_Tenx_{tenx_cell}_rep_{N_rep}_pen_{penelty}_C_{C_value}_solver_{solver_mode}'
                            print(param)
                            para_list.append(param)
len(para_list)

In [ ]:
len(para_list)

In [ ]:
df_pred_all = pd.DataFrame(index = list(adpatch.obs_names))

for para in para_list:
    #print(para)
    import pickle
    with open(f'data/predict_results_all/{para}.pckl', 'rb') as f:
        x = pickle.load(f)
    df_pred_all[para] = x['rep_0'].idxmax(axis = 1)
    
df_pred_all.to_csv('results/df_predict_all_param.csv')  

In [ ]:
label_summary_all = pd.DataFrame(index = df_pred_tops.index)

label_summary_all['best_fit'] = np.nan
label_summary_all['best_fit_score'] = np.nan

label_summary_all['2nd_fit'] = np.nan
label_summary_all['2nd_fit_score'] = np.nan


all_cells = df_pred_all.index.tolist()
for cell in all_cells:
    counts_info = df_pred_all.loc[cell,:].value_counts()
    label_summary_all.loc[cell,'best_fit'] = counts_info.index.tolist()[0]
    label_summary_all.loc[cell,'best_fit_score'] = counts_info[0]/sum(counts_info)
    label_summary_all.loc[cell,'2nd_fit'] = counts_info.index.tolist()[(int(len(counts_info)>1))]
    label_summary_all.loc[cell,'2nd_fit_score'] = counts_info[(int(len(counts_info)>1))]/sum(counts_info)
label_summary_all

In [ ]:
hist(label_summary_all['best_fit_score'])

In [ ]:
label_summary_all['best_fit_score'].mean()

In [ ]:
label_summary_all['best_fit_score'].std()

In [ ]:
label_summary_all

In [ ]:
label_summary_all.to_csv('data/SCCAF_projection_retina_all.csv')